Free Parking Model predicts the availability of free parking at a person’s workplace. It is applied for people who work in zones that have parking charges

- only workers that have a usual work location outside their home and that is in a zone that has paid parking
    - PRKCST > 0
    - workers with usual work location

In [1]:
import os
import toml
import pandas as pd
import validation_data_input
import numpy as np
import plotly.express as px

# to show plotly figures in quarto HTML file
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook_connected"

# get shared data
%store -r validation_data

config = toml.load(os.path.join(os.getcwd(), 'validation_configuration.toml'))

In [2]:
# read data

#per_data = validation_data_input.get_persons_data()
per_data = validation_data.persons_data_uncloned.copy()
land_use = validation_data.land_use[['zone_id', 'TAZ', 'county_id']].copy()
# delete shared data
del validation_data


per_data = per_data.merge(land_use, how="left",left_on='home_zone_id',right_on='zone_id')

per_data = per_data.loc[per_data['pemploy']<3]

- number of people who work in zones that have parking charges

In [3]:
per_data.groupby(['source'])['person_weight'].sum().reset_index()

,source,person_weight
0,model results,1.982128e+06
1,survey data,2.245197e+06
2,unweighted survey,3.892000e+03


## Work From Home

In [4]:
df_plot = per_data.groupby(['source','work_from_home'])['person_weight'].sum().reset_index()

df_plot['percentage'] = df_plot.groupby(['source'], group_keys=False)['person_weight']. \
    apply(lambda x: x / float(x.sum()))
df_plot['source'] = df_plot['source'].astype(pd.CategoricalDtype(['model results', 'survey data', 'unweighted survey']))

fig = px.bar(df_plot.sort_values(by=['source']), x="work_from_home", y="percentage", color="source",
             barmode="group",template="simple_white",
             title="Work From Home")
fig.update_layout(height=400, width=700, font=dict(size=11), yaxis_tickformat = '.0%')
fig.show()

In [5]:
# auto ownership in Income groups
def plot_segments(df:pd.DataFrame, var:str, title_cat:str,sub_name:str):
    # print(f"n=\n"
    #       f"{df.loc[df['source']=='model results',var].value_counts()[df[var].sort_values().unique()]}")
    df_plot = df.groupby(['source',var,'work_from_home'])['person_weight'].sum().reset_index()
    df_plot['percentage'] = df_plot.groupby(['source',var], group_keys=False)['person_weight'].\
        apply(lambda x: 100 * x / float(x.sum()))

    fig = px.bar(df_plot, x="work_from_home", y="percentage", color="source",
                 facet_col=var, barmode="group",template="simple_white",
                 title="Work From Home by "+ title_cat)
    fig.for_each_annotation(lambda a: a.update(text = sub_name + "=<br>" + a.text.split("=")[-1]))
    fig.update_xaxes(title_text="n of work from home")
    fig.update_layout(height=400, width=800, font=dict(size=11))
    fig.show()

In [6]:
plot_segments(per_data,'county_id','County', 'County')

In [7]:
print("end")

end
